In [ ]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler

from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import RandomForestRegressor

import gc 

# load Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")

# reduce memory 

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

train = reduce_memory_usage(train)
test = reduce_memory_usage(test)
gc.collect()

In [ ]:
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

# install pycaret 

In [ ]:
from IPython.display import clear_output
!pip install pycaret --user
clear_output()

# import libraries

In [ ]:
import numpy as np
import pandas as pd
import pycaret
from pycaret.regression import *
from sklearn.model_selection import train_test_split

# global variables

In [ ]:
TARGET = "target"
SESSION_ID = 2021
SEED = 42
FEATURE = "feature"
EXPERIMENT_NAME = 'tabular_prediction'
N_SPLITS = 5

# predict test data

In [ ]:
test_pred = np.zeros(test.shape[0])

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=False, random_state=SEED)

for fold, (trn_idx, val_idx) in enumerate(skf.split(X=train, y=train[TARGET])):
    setup(data = train.iloc[val_idx], 
      target = TARGET, 
      session_id=SESSION_ID, 
      experiment_name=EXPERIMENT_NAME,
      silent=True,
      normalize=True,
      transformation=True
     )
    
    lightgbm = create_model('lightgbm')
    
#     lgbm_params = {
# #                 'num_leaves': np.arange(10,200,10),
#                 'max_depth': np.arange(1,10,1),
#                 'learning_rate': [0.1,0.05,0.001,0.0005,0.0001,0.00005,0.00001],
# #                 'n_estimators':np.arange(100,2000,100),
# #                 'min_child_samples':np.arange(10,1000,10),
# #                 'subsample_freq':np.arange(0,1000,10)
#              }

#     tuned_lightgbm = tune_model(lightgbm, custom_grid = lgbm_params)
    tuned_lightgbm = tune_model(lightgbm)
    
    final_lightgbm = finalize_model(tuned_lightgbm)
    
    test_pred += predict_model(final_lightgbm, data=test)["Label"] / N_SPLITS
    gc.collect()

# submission

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")
submission.target = test_pred
submission.to_csv("submission.csv",index=False)
submission.head(10)